In [ ]:
!wget --load-cookies /tmp/cookies.txt "https://drive.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://drive.google.com/uc?export=download&id=0B7XkCwpI5KDYNlNUTTlSS21pQmM' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=0B7XkCwpI5KDYNlNUTTlSS21pQmM" -O GoogleNews-vectors-negative300.bin.gz && rm -rf /tmp/cookies.txt

--2023-09-18 13:32:23--  https://drive.google.com/uc?export=download&confirm=t&id=0B7XkCwpI5KDYNlNUTTlSS21pQmM
Resolving drive.google.com (drive.google.com)... 142.251.162.139, 142.251.162.113, 142.251.162.101, ...
Connecting to drive.google.com (drive.google.com)|142.251.162.139|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0g-8s-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/gdf94ca2ch1eingcl08g91oa7uthr7qp/1695043875000/06848720943842814915/*/0B7XkCwpI5KDYNlNUTTlSS21pQmM?e=download&uuid=29dc569d-f412-47aa-9e9e-6bfc1bad9117 [following]
--2023-09-18 13:32:23--  https://doc-0g-8s-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/gdf94ca2ch1eingcl08g91oa7uthr7qp/1695043875000/06848720943842814915/*/0B7XkCwpI5KDYNlNUTTlSS21pQmM?e=download&uuid=29dc569d-f412-47aa-9e9e-6bfc1bad9117
Resolving doc-0g-8s-docs.googleusercontent.com (doc-0g-8s-docs.googleusercontent.com)... 173.194.212.132, 2607:

In [ ]:
!gzip -d GoogleNews-vectors-negative300.bin.gz

In [ ]:
import pandas as pd
from gensim.models import KeyedVectors

model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)
train = pd.read_csv('train.txt', sep='\t')
valid = pd.read_csv('valid.txt', sep='\t')
test = pd.read_csv('test.txt', sep='\t')
train.head()

,category,title
0,b,Bouygues confirms improved offer for Vivendi's...
1,b,"US stocks end off highs, Dow up for week; gold..."
2,e,'Chef' Director: The Creative Driving Force Be...
3,b,South African labor minister to meet with stri...
4,e,Kaley Cuoco and her Big Bang Theory co-stars h...


In [ ]:
import numpy as np

df = pd.concat([train, valid, test])
vecs = []
for sentence in df['title']:
  sentence_vec = []
  for word in sentence.split():
    try:
      sentence_vec.append(model[word])
    except KeyError:
      sentence_vec.append(np.zeros(300))
  vecs.append(np.mean(sentence_vec, axis=0))
vecs = np.array(vecs)
vecs.shape

(13340, 300)

In [ ]:
X_train = vecs[:len(train)]
X_valid = vecs[len(train):-len(test)]
X_test = vecs[-len(test):]
y = df['category'].replace({'b': 0, 't': 1, 'e': 2, 'm': 3})
y_train = y[:len(train)]
y_valid = y[len(train):-len(test)]
y_test = y[-len(test):]
print(X_train.shape, X_valid.shape, X_test.shape, y_train.shape, y_valid.shape, y_test.shape)

(10672, 300) (1334, 300) (1334, 300) (10672,) (1334,) (1334,)


In [ ]:
from joblib import dump

dump(X_train, 'X_train.joblib')
dump(X_valid, 'X_valid.joblib')
dump(X_test, 'X_test.joblib')
dump(y_train, 'y_train.joblib')
dump(y_valid, 'y_valid.joblib')
dump(y_test, 'y_test.joblib')

['y_test.joblib']